#  Séries de Fourier sinus-cosinus

## Intro

### settings

In [ ]:
%reset -f

In [ ]:
!git clone https://github.com/vincentvigon/assets_signal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

np.set_printoptions(linewidth=500,precision=3,suppress=True)
plt.style.use("default")

### Summary


A Fourier series is the decomposition of a signal (=regular function) defined on a time-interval $[0,T]$ as an infinite sum of elementary waves. These waves can be either

* sinus/cosinus functions
* only cosinus
* complex exponential functions


You have to distinguish:

* Fourier series: time is $[0,T]$
* Fourier transform: time is $\mathbb R$. Too theoric for our purpose.
* Discrete Fourier transform: time is a finite set $\{0,1,...,N-1\}$. We will see this pratical case on the next chapter.


### Technical words


After reading this chapter,  you might be familiar with all this terms (if not, re-read this chapter!)

* Fourier series
* Contiuous time/discrete time
* SinCos wave basis, exponential wave basis
* Period, frequency
* piecewise-smooth function
* dot product, hermitian product, orthogonality, orthonormality
* Gipps phenomenum, absolute convergence
* Hermitian symetry
* Spectrum, half-spectrum, amplitude-spectrum, fourier coefficient, coordinates
* signal approximation, signal filtration, signal compression
* Low-pass filters, high-pass filters, band-pass filters
* Perfect sampling
* Orthogonal polynomial, Legendre polynomial, gauss point
* Hilbert basis, Plancherel-Parseval identity, isometry
* Energy of a signal

## Signals

### Definition


 Continuous-time signals are defined on an time interval $[0,T]$. Most famous signals are sounds.  Sounds are produced by the variations of  the air pressure.

In [ ]:
#time
t=np.linspace(0,2,2000)
#signal
signal0=(np.sin(4*2*np.pi*t)+0.5*np.sin(7*2*np.pi*t))*t**2

fig,ax=plt.subplots()
ax.plot(t,signal0)
ax.set_xlabel("time")
ax.set_ylabel("pressure");

### Periodic signal

A periodic function repeats itself after a while.

***Definition:***

* A function $f$ as period $T>0$ when $f(t + T) = f(t)$ for all $t$.
* **The** period of a function $f$ is the smallest $T>0$ such that $f$ is $T$-periodic
* The frequency of a periodic function is the number of complete cycles that can occur per second. So it is the inverse of the period.

Remarks:

* a $T$-periodic function is also $nT$-periodic for $n=2,3,...$.
* One can say that a non-periodic has a period $0$ and a frequency $\infty$ but I prefer I prefer to say that it is a non-periodic function
* It is natural and usefull to say that the constant function has a frequency $0$.
* The unit for frequency is the 'Hertz', abreviated by 'Hz'. Physicaly: Hz=$s^{-1}$.


In [ ]:
"A periodic signal (but only ploted on a bounded interval)"
signal1=np.sin(4*2*np.pi*t)+0.5*np.sin(8*2*np.pi*t)
fig,ax=plt.subplots()
ax.plot(t,signal1);

***To you:***  Give the period and the frequency of the following functions:

*  $(1\heartsuit)$ $t \to \sin(4*2\pi*t)$
* $(1\heartsuit)$ $t \to \sin(4*t)$
*  $(1\heartsuit)$ $t \to \sin(3*2\pi*t)+\sin(4*2\pi*t)$.  Plot this one, in such a way we see its periodicity.
*  $(1\heartsuit)$ $t \to \sin(3*2\pi*t)*\sin(4*2\pi*t)$.  Plot this one, in such a way we see its periodicity.
* $(1\heartsuit)$  $t\to \cos\big(\sin( \pi t/20 )\,/\,\sin(\pi t/30)\big)$.

* $(1\heartsuit)$ $t \to \sin(4*2\pi*t)+\sin(7*t)$


Help: if $\forall t$, $f_1(t+T_1)=f(t)$ and $f_2(t+T_2)=f_2(t)$, what can we say about $(f_1+f_2)(t+T)$ when $T=k_1T_1=k_2T_2$ for some integer constant $k_1$ and $k_2$?

## SinCos decomposition

During all this section, we will work on a the time-interval $[0,T]$ with $T=2$ seconds. A function defined on this interval can be either:

* a part of a periodic signal with period $T$ (so we implicitly prolongate it periodicaly),
* or just a signal taken on a finite interval (ex: the  recording of a sound).



In [ ]:
"""we define our time interval,
and its discretisation"""
T=2
nb_points=200
t=np.linspace(0,T,nb_points,endpoint=False)
"""the amount of time between two points """
step=T/nb_points

### Regularity of signals



***Definition:***  A function $f$ is *piecewise-smooth* (=lisse par morceaux) when $f$ and $f'$ are piecewise continuous.

This is the regularity satisfied by all signals we meet in the nature. So from now: **the word 'signal' means 'piecewise-smooth function'**.


Here is two additionnal regularities which can occurs or not:

* A Signal $f$ can be continuous (no jump).
* A Signal $f$ can be  continuous and satisfying $f(0)=f(T)$.

***To you:*** What is the advantage of the condition $f(0)=f(T)$ when we want to prolongate $f$ periodicaly on $\mathbb R$?



In [ ]:
Image("assets_signal/lisseParMorceaux.png")

In [ ]:
"""example of a non-continuous signal.
Jumps are the vertical lines"""
def square_signal(t):
    posi=(np.sin(4*np.pi*t)>0)
    f=np.empty(len(t))
    f[posi]=1
    f[~posi]=-1
    return f

plt.plot(t,square_signal(t));

***To you:*** $(1\heartsuit)$ Change the above drawing so that the vertical parts deseapear (more realistic for jumps). Help: it is very simple, just change the style!


***To you:***
* $(2\heartsuit)$ Add an argument to `square_signal()` which can rules the period.
* $(3\heartsuit\flat)$  Create a function `triangle_signal()` that produce teeth of a saw. It is an example of a contiuous signal.

### A dot product


For $f$ and $g$ signals on $[0,T]$, we define:
$$
 \mathtt{dot}(f,g) = \frac 2 T \int_0^T f(t) g(t) \, dt
$$

This is a dot-product (=produit scalaire), which means that satisfy the folloing properties:
* Symmetry: $\mathtt{dot}(f,g)=\mathtt{dot}(g,f)$
* Bi-linearity: $f\to \mathtt{dot}(f,g)$ and $g\to \mathtt{dot}(f,g)$ are linear.
* Positive-define:
$$
\mathtt{dot}(f,f)\geq0 \qquad \text{and} \qquad   \mathtt{dot}(f,f)= 0\Leftrightarrow f=0
$$


***To you:*** $(2\heartsuit)$ Check  that these 3 properties are satisfied by $\mathtt{dot}()$. For the last one, you need to use the implicit regularity of signals $f$.



In [ ]:
"""the informatic version of the scalar product.
The integral is approximate by a sum"""
def dot(f,g):
    return 2*np.sum(f*g)/len(f)
""" if we denote by 'step' the interval between two discretization points, we have
         2*np.sum(f*g)/len(f) = 2/T*np.sum(f*g)*step
""";

***To you:*** In the function above, the integral is approximate using the rectangle method. Do you know other methods $(1\heartsuit)$? Probably other methods are better, but please, do not change the above function. We have a good reason to use the rectangle method here (see later on).

In [ ]:
Image("assets_signal/rectangleMethod.png")

In [ ]:
"let's test the dot product"
f=np.sin(t)
g=np.ones_like(t)
dot(f,g)

***To you:*** $(1\heartsuit)$ The above numeric value is  an approximation of ...

### sin-cos family


We define now some special functions on $[0,T]$:
\begin{align*}
\sin_n(t)& =\sin(2\pi \frac {nt} T),  \qquad \forall n\geq 1 \\
\cos_{n}(t) & =\cos(2\pi \frac {nt} T),  \qquad \forall n\geq 1
\end{align*}

We want to stack all this functions  in one family,  together with the constant function $t \to  \frac 1 {\sqrt 2}$. So we write:
\begin{align*}
sc_0(t)&= \frac 1 {\sqrt 2}\\
sc_{2i-1}(t)& =\sin_i(t),  \qquad \forall i\geq 1 \\
sc_{2i}(t) & =\cos_i(t),  \qquad \forall i\geq 1
\end{align*}


The infinite familly $(sc_n)_{n\in \mathbb N} $ is called the sinCos family. As we will check, it is an orthonormal family for our dot product.


In [ ]:
"""The begin of the sinCos family, stacked in a matrix.
The n-th line is the discretization of sc_n(t)"""

def compute_sinCos_basis(t,T,M):
    """
    Args:
         t (1d-array) : time discretized
         T (int) : Duration
         M (int) : number of different frequencies, so the size of the basis is 2N+1
    """

    basis_sc=np.empty([2*M+1,len(t)])
    basis_sc[0]=np.ones_like(t)/np.sqrt(2)
    for i in range(1,M+1):
        basis_sc[2*i-1,:]=np.sin(i*2*np.pi*t/T)
        basis_sc[2*i,:]=np.cos(i*2*np.pi*t/T)

    return basis_sc

In [ ]:
basis_sc=compute_sinCos_basis(t,T,10)

for i in range(8):
    plt.plot(t,basis_sc[i,:])

In [ ]:
"""compute some dot-product to check the orthonormality"""
nb=10
many_dot_products=np.empty([nb,nb])

for i in range(nb):
    for j in range(nb):
        many_dot_products[i,j]=dot(basis_sc[i,:],basis_sc[j,:])

print(many_dot_products)

***To you:*** $(2\heartsuit)$ The previous code is pedagogic but not efficient. Make it faster: suppress the function `dot` and suppress the double loop on `i` and `j` to recompute the matrix above.  Help: use the matrix-product `@`. You can also play to note all places where the code can be optimized in the following.

### sin-cos approximation


A signal $f$ can be decompose as an infinite sum of elements of the sinCos family:
$$
f(t)=\sum_{n\in \mathbb N} a_n \, sc_n(t)
$$
where, from the orthonormality, the coordinates $a_n$ have a simple expression:
$$
a_n= \mathtt{dot}(f,sc_n)
$$

In particular, a signal can be approximate by a finite sum of elements of the sinCos family:
$$
f(t) \simeq\sum_{n=0}^{2M+1} a_n \, sc_n(t)
$$

In [ ]:
"""here is the function to decompose"""
f=(t-1)**2
plt.plot(t,f);

In [ ]:
"""the coordinate of f in the sinCos family.
Because of the orthonormality, this coordinate are very easy to compute"""
coordinates=np.empty(len(basis_sc))

for i in range(len(basis_sc)):
    coordinates[i]=dot(f,basis_sc[i,:])

plt.plot(range(len(basis_sc)),coordinates,".");

***To you:*** $(2\heartsuit)$ Explain the particularity of the plot above.

In [ ]:
"""each line of the following matrix represent an approximation of the signal f """
approximations=np.empty([len(basis_sc),nb_points])
for i in range(len(basis_sc)):
    coor_troncated=coordinates.copy()
    coor_troncated[i:]=0
    approximations[i,:]=coor_troncated@basis_sc
    #ou plus proche des maths:
    #approximations[i,:]=basis_sc @ coor_troncated[:,None]


approximations

In [ ]:
"""let's plot some of the approximations"""
nb_plots=4
fig,axs=plt.subplots(nb_plots,figsize=(5,nb_plots*3),sharex=True)
fig.tight_layout()

for i in range(nb_plots):
    axs[i].plot(t,approximations[2*i,:])
    axs[i].plot(t,f)
    axs[i].set_title("nb sinCos="+str(2*i))

***To you:*** $(1\heartsuit)$ Why did we only plot the approximations made with an even number of sinCos?


***To you:***  Make the error plot, that illustrate how the error between the function `f` and its approximations decrease. To compute the error use:

* $(2\heartsuit)$ the $L_2$ norm with the help of `dot()`
* $(2\heartsuit)$ the $L_\infty$ norm  with the help of `np.max()`

## Gipps and theory

### Make general code

We wrap all the previous code into a class, that we will use several times in the sequel. The purpose is also to show you how objets allow to well organize your code.

In [ ]:
class Decomposer:

    def __init__(self,t,basis,dot_fn,dtype=np.float64):

        self.t=t
        self.basis=basis
        self.dot=dot_fn
        self.dtype=dtype


    def check_ortho(self):

        nb=len(self.basis)


        res=np.empty([nb,nb],self.dtype)
        for i in range(nb):
            for j in range(nb):
                res[i,j]=self.dot(self.basis[i,:],self.basis[j,:])
        print(res)


    def compute_coordinates(self,f,plotThem=False):


        coordinates=np.empty(len(self.basis),dtype=self.dtype)

        for i in range(len(self.basis)):
            coordinates[i]=self.dot(f,self.basis[i,:])

        if plotThem:
            plt.plot(range(len(self.basis)),self.OUT_coordinates,".")

        return coordinates



    def compute_approximations(self,f,approx_indexes,plotThem=False):


        assert max(approx_indexes)<=len(self.basis), "approximation index can not be greater than the size of the basis"

        coordinates=self.compute_coordinates(f)


        """each line of the folowing matrix represent an approximation f_N of the signal f """
        approximations=np.empty([len(approx_indexes),len(self.t)],dtype=self.dtype)

        for i,j in enumerate(approx_indexes):
            coor_troncated=coordinates.copy()
            coor_troncated[j:]=0
            approximations[i,:]=coor_troncated@self.basis


        if plotThem:
            nb=len(approx_indexes)
            if nb<=1: nb=2 # to avoid a bug when we write axs[i]
            fig,axs=plt.subplots(nb,1,figsize=(8,nb*2))


            for i in range(len(approx_indexes)):
                axs[i].plot(self.t,f)
                axs[i].plot(self.t,approximations[i,:])
                axs[i].set_title("sum of "+str(approx_indexes[i])+" terms")

            fig.tight_layout()


        return approximations


***To you:***  Why the previous class can not be replaced by a simple function?


In [ ]:
T=2
t=np.linspace(0,T,2000,endpoint=False)
basis_sc_small=compute_sinCos_basis(t,T,3)

decomposer=Decomposer(t,basis_sc_small,dot)

In [ ]:
decomposer.compute_approximations(t**2,[1,5,7],True);

***To you:*** $(1\heartsuit)$ Make work the whole pipeline, with the function to approximate $f(t)=t^2$. Note how approximations becomes bad. The explanation comes in the next section.  

***To you:*** $(3\heartsuit)$ Add a method `compute_L2_error()` and `compute_Loo_error()`,  which allow to incorporate your previous work into the `Decomposer`.

### Gipps phenomenum





In [ ]:
f_discont=square_signal(t)
plt.plot(t,f_discont);

In [ ]:
basis_sc_big=compute_sinCos_basis(t,T,100)
basis_sc_big.shape

In [ ]:
decomposer=Decomposer(t,basis_sc_big,dot)

In [ ]:
decomposer.compute_approximations(f_discont,[10,50,100,150,200],True);

***To you:*** $(2\heartsuit)$ Show with curves than the above approximations converge or not converge according to the norm we consider. You can use for this the `Decomposer` that you have modified.


***To you:*** $(1\heartsuit)$ So: discontinuities implies bad approximations. But does the function $t\to t^2$ you tried before is discontinuous?

### Summary in one theorem



The next theorem summarize all that we have seen:

***The 3 points theorem, sinCos version:***

* The familly $(sc_n)$ is orthonormal for  $\mathtt{dot}()$
* Signals $f$ can be writed:
\begin{align*}
f(t) &= \sum_{n\in \mathbb N} a_n \, sc_n(t) \\
&=  \frac{a_0}{\sqrt 2} +  \sum_{i\geq 1} a_{2i-1} \sin(2\pi \frac{it}{T}) +a_{2i} \cos(2\pi \frac{it}{T})
\end{align*}

    * the sum converges for every $t$ where $t\to f(t)$ is continuous
    * when $f$ is continuous and $f(0)=f(T)$ the convergence is uniform.

* The coordinates $a_n$ are easy to compute $a_n=\mathtt{dot}(f,sc_n)$.



***Comments:***

* The first point is often proved using product-to-sum trigonometric identities e.g:
$$
2\cos(\theta)\cos(\phi)=\cos(\theta-\phi)+\cos(\theta+\phi)
$$
But we will use a realy simpler argument later on.
* The second point is the difficult part. It means that our sinCos family is a sort of basis for signals. Actualy, is we chose to work on the $L^2$ context, it is an Hilbert basis (see later on).
* The coordinates $a_n$ are called 'sinCos Fourier coefficients', sometimes we denote them by $a_n(f)$ or $\hat f(n)$ or $Fourier[f](n)$.
* Let's check the formula that allows to compute the Fourier coefficients:
\begin{align*}
  & \mathtt{dot}\Big(f,sc_n\Big) \\
=& \mathtt{dot}\Big( \sum_{m} a_m \, sc_m(t) \ , \ sc_n   \Big)  \\
=& ...\\
=&a_n
\end{align*}

***To you:***

* $(2\heartsuit)$ Complete  the computation above. If you want to be rigourous, you can restrict yourself to the case where the serie converge uniformly.
* $(1\heartsuit)$ Why, when $f$ has discontinuities, it is impossible that the serie converge uniformly?


## Only-cos decomposition

Now we work on a symetric time interval $[-\frac T 2 , +\frac T 2]$. If we have in mind periodic functions, we only decay the window where we observe these functions. The new dot product is of course:
$$
\mathtt{dot} (f,g) = \frac 2 T \int_{-\frac T 2}^{+\frac T 2} f(t) g(t) \, dt
$$
The informatic version of this dot-product do not change at all!


A even function $f_{even}$ can be decomposed only with the cosinus terms: simply because the coordinates relative to sinus are zeros:
$$
\frac 1 T \int_{-\frac T 2}^{+\frac T 2} f_{even}(t) \sin_n(t) \, dt=0
$$
Let's observe this.

In [ ]:
"""we define our time interval,
and its discretisation"""
T=2
t=np.linspace(-T/2,T/2,200,endpoint=False)

In [ ]:
def compute_cos_basis(t,T,nb_freq):
    basis=np.empty([nb_freq+1,len(t)])
    basis[0]=np.ones_like(t)/np.sqrt(2)
    for i in range(nb_freq):
        basis[i+1,:]=np.cos((i+1)*2*np.pi*t/T)

    return basis

In [ ]:
basis_cos=compute_cos_basis(t,T,10)

for i in range(7):
    plt.plot(t,basis_cos[i,:])

In [ ]:
f_even=t**2
plt.plot(t,f_even);

In [ ]:
decomposer=Decomposer(t,basis_cos,dot)

In [ ]:
decomposer.compute_approximations(f_even,[2,5,7,11],True);

This simple observation give us an idea to create a decomposition for signals defined on $[0,\frac T 2 ]$ with only cosinus. This decomposition is simpler, and often used in practice (ex: JPEG compression).



***To you:*** $(2\heartsuit)$ Explain this idea, and explain why this only-cos decomposition is  less subjet to the Gipps phenomenum.


***To you:*** $(4\heartsuit\flat)$ Make a program that compute the only-cos-decomposition of signals defined on $[0,\frac T 2 ]$. To not avoid useless computations, you need to modify the previous codes. Test your program with $f(t)=t$ defined on $[0,\frac T 2 ]$. Be careful, this is very short, but tricky.


